In [1]:
import pandas as pd
import numpy as np
import os
import pickle
from scipy.stats import norm
from scipy.optimize import fsolve
from scipy import interpolate
from scipy.interpolate import interp1d
from scipy.integrate import quad
from scipy.optimize import least_squares
from datetime import datetime
from math import exp, sqrt, log

In [2]:
## read OIS excel sheet 
file_path = os.path.abspath("data/IR Data.xlsx")
df_ois = pd.read_excel(file_path, sheet_name="OIS", usecols=[0,1,2])

# convert tenor to years 
tenor_mapping = {
    '6m': 0.5,
    '1y': 1,
    '2y': 2,
    '3y': 3,
    '4y': 4,
    '5y': 5,
    '7y': 7,
    '10y': 10,
    '15y': 15,
    '20y': 20,
    '30y': 30
}

df_ois['Tenor'] = df_ois['Tenor'].map(tenor_mapping)
df_ois['Tenor_Delta'] = df_ois['Tenor'].diff().fillna(0.5)
df_ois


,Tenor,Product,Rate,Tenor_Delta
0,0.5,OIS,0.00250,0.5
1,1.0,OIS,0.00300,0.5
2,2.0,OIS,0.00325,1.0
3,3.0,OIS,0.00335,1.0
4,4.0,OIS,0.00350,1.0
5,5.0,OIS,0.00360,1.0
6,7.0,OIS,0.00400,2.0
7,10.0,OIS,0.00450,3.0
8,15.0,OIS,0.00500,5.0
9,20.0,OIS,0.00525,5.0


In [3]:
## read IRS excel sheet 
irs_df = pd.read_excel(file_path, sheet_name="IRS", usecols=[0,1,2])
irs_df  

,Tenor,Product,Rate
0,6m,LIBOR,0.0250
1,1y,IRS,0.0280
2,2y,IRS,0.0300
3,3y,IRS,0.0315
4,4y,IRS,0.0325
5,5y,IRS,0.0330
6,7y,IRS,0.0350
7,10y,IRS,0.0370
8,15y,IRS,0.0400
9,20y,IRS,0.0450


In [4]:
## read libor discount curve & ois pckl files 
libor_curve = pickle.load(open("data/libor_discount_curve.pkl", "rb"))
ois_discount_curve = pickle.load(open("data/ois_discount_curve.pkl", "rb"))

In [5]:
## read forward swap rates based on ann's codes
file_path_swap = os.path.abspath("data/swap_rates.xlsx")
forward_swap_rates = pd.read_excel(file_path_swap)
forward_swap_rates.rename(columns={'Unnamed: 0':'Expiry'}, inplace=True)
forward_swap_rates = forward_swap_rates.set_index('Expiry')
forward_swap_rates.columns.name = 'Tenor'

forward_swap_rates

Tenor,1,2,3,5,10
Expiry,,,,,
1,0.032007,0.033259,0.034011,0.035255,0.038428
5,0.039274,0.040075,0.040072,0.041093,0.043634
10,0.042189,0.043116,0.044097,0.046249,0.053458


In [6]:
r = np.arange(0.5, 20.5, 0.5)
interpolated_libors = libor_curve(r)
interpolated_ois = ois_discount_curve(r)
interpolated_ois_df = pd.DataFrame({
    'Tenor': r,
    'OIS_Discount_Factor': interpolated_ois,
    'Forward_Libor_Rates': interpolated_libors
    })

interpolated_ois_df = interpolated_ois_df.set_index('Tenor')
interpolated_ois_df.head()

,OIS_Discount_Factor,Forward_Libor_Rates
Tenor,,
0.5,0.998752,0.987654
1.0,0.997009,0.972577
1.5,0.995270,0.957378
2.0,0.993531,0.942179
2.5,0.991773,0.926330


In [7]:
## daily compounded overnight rate 
def calc_compounded_on_rate(f_t, delta_T):
    """
    calculate the daily compounded return based on the overnight rate

    Args:
    f_t(float): overnight rate
    T(float): tenor

    Returns:
    compounded overnight rate
    """
    return 1 / ((1 + f_t/360) ** (360 * delta_T))

## discount factor 
def calc_discount_factor(prev_df, f_t, delta_T):
    """
    calculate the discount factor for a given tenor

    Args:
    prev_df(float): previous discount factor
    f_t(float): overnight rate
    delta_T(float): time difference

    Returns:
    discount factor
    """
    return prev_df * calc_compounded_on_rate(f_t, delta_T)

## forward libor rate 
def calc_forward_libor(prev_df, current_df, delta_T):
    """
    calculate the forward libor rate for a given tenor

    Args:
    prev_df(float): previous discount factor
    current_df(float): current discount factor
    delta_T(float): time difference

    Returns:
    forward libor rate
    """
    return (1/delta_T) *(prev_df/current_df - 1)

In [8]:
# ## calculation of daily comoounded overnight rate and discount factor - Junhao's codes

# previous_flt_payouts = [] # this is used to easily calculate the previous flt_payouts 
# previous_discount_factors = []

# DF_name = "Discount Factor"

# for index, row in df_ois.iterrows():
#     f = -1 # temporary place holder
#     current_discount_factor = -1
#     if index == 0: 
#         # T = 0.5
#         f = ( (row['Tenor'] * row['Rate'] + 1) ** (1/180) - 1 ) * 360
#         current_discount_factor = calc_discount_factor(1, f, 0.5) # no previous discount factor, set to 1 for multiplying
#     elif index == 1: 
#         # T = 1;
#         first_row = df_ois.iloc[0]

#         # note: discount_factor Do(0,1Y) is not here because its cancelled out, so no need to include, actually can just manually rearrange
#         f = fsolve(
#             lambda f:
#                ( (1 + first_row['f']/360) ** 180 ) * ( (1 + f/360) ** 180) # PV flt
#                - 1.003 # PV fix 
#             ,
#             x0 = first_row['f']
#             )

#         current_discount_factor = calc_discount_factor(first_row[DF_name], f, 0.5);
#         current_flt_payout = current_discount_factor * (( ((1 + first_row['f']/360) ** 180) * ((1 + f/360)** 180)) - 1 )

#         # saving it to use later
#         previous_flt_payouts.append(current_flt_payout)
#         previous_discount_factors.append(current_discount_factor)
#     else:
#         previous_row = df_ois.iloc[index-1];
#         N = row['Tenor'] - previous_row['Tenor']
#         # fsolve is trying to find overnight rate f -> using 0 = PV_flt - PV_fix
#         f = fsolve(
#             lambda f: 
#             np.sum(previous_flt_payouts) + calc_discount_factor(previous_row[DF_name], f, N) * ((1 + f/360) ** (360 * N) - 1) # pv of flt = sum of previous flt payout + current flt payout 
#             - (np.sum(previous_discount_factors) + calc_discount_factor(previous_row[DF_name], f, N)) * row['Rate'], # pv of fix = [sum of previous discount factor + current discount factor] * ois_par_swap_rate
#             x0= previous_row['f']
#         )
        
#         current_discount_factor = calc_discount_factor(previous_row[DF_name], f, N)
#         current_flt_payout = current_discount_factor * ((1 + f/360) ** (360 * N) - 1)

#         previous_flt_payouts.append(current_flt_payout)
#         previous_discount_factors.append(current_discount_factor)

#     df_ois.at[index, 'f'] = f;
#     df_ois.at[index,DF_name] = current_discount_factor
# df_ois

In [9]:
## black 76 model 
class black_76_model:
    def __init__(self, F: float, K: float, sigma: float, discount_factor: float, T: float):
        self.F = F
        self.K = K
        self.sigma = sigma
        self.discount_factor = discount_factor
        self.T = T
        self.d1 = self.calc_black_scholes_d1()
        self.d2 = self.calc_black_scholes_d2()

    def calc_black_scholes_d1(self) -> float:
        sigma_sqrt_time = self.sigma * np.sqrt(self.T)
        return (np.log(self.F / self.K) + (np.power(self.sigma, 2)/2) * self.T ) / sigma_sqrt_time
    
    def calc_black_scholes_d2(self) -> float:
        sigma_sqrt_time = self.sigma * np.sqrt(self.T)
        return self.d1 - sigma_sqrt_time
    
    def blackscholes_call(self):
        return self.discount_factor*(self.F*norm.cdf(self.d1) - self.K*norm.cdf(self.d2))

    def blackscholes_put(self):
        return self.discount_factor*(self.F*norm.cdf(-self.d1) + self.K*norm.cdf(-self.d2))

In [10]:
## SABR model
def SABR(F, K, T, alpha, beta, rho, nu):
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom

    return sabrsigma     

In [11]:
# ## calculation of forward swap rates - still wrong so used Ann's swap rates excel instead 

# # define expiries and tenors
# expiries = [1, 5, 10]  # Row indices (expiry in years)
# tenors = [1, 2, 3, 5, 10]  # Column indices (tenor in years)

# # initialize the forward swap rate matrix
# forward_swap_matrix = np.zeros((len(expiries), len(tenors)))
# print(forward_swap_matrix)

# def calc_forward_swap_rates(expiry, tenor, interpolated_ois_df, fixed_leg_freq, float_leg_freq):
#     swap_end = expiry + tenor # swap duration 
#     # payment dates 
#     float_leg_payment = np.arange(
#         expiry+float_leg_freq, # start from the next payment date
#         swap_end+float_leg_freq, # end at the last payment date
#         float_leg_freq # payment freq - semi-annual
#     )

#     fixed_leg_payment = np.arange(
#         expiry+fixed_leg_freq,
#         swap_end+fixed_leg_freq,
#         fixed_leg_freq
#     )
    
#     ois_discount_values = interpolated_ois_df.loc[float_leg_payment, 'OIS_Discount_Factor'].values
#     forward_libor_values = interpolated_ois_df.loc[float_leg_payment, 'Forward_Libor_Rates'].values

#     # pv of fixed and float leg
#     fixed_leg_pv = np.sum(fixed_leg_freq * ois_discount_values) 
#     float_leg_pv = np.sum(float_leg_freq * ois_discount_values * forward_libor_values)

#     return float_leg_pv / fixed_leg_pv

In [12]:
# for i, expiry in enumerate(expiries):
#     for j, tenor in enumerate(tenors):
#         forward_swap_matrix[i, j] = calc_forward_swap_rates(expiry, tenor, interpolated_ois_df, 0.5, 0.5)
        
# forward_swap_matrix
        

In [13]:
## read sabr params data and pre-process
file_path_sabr = os.path.abspath("data/sabr_calibrated.xlsx")
df_sabr_alpha = pd.read_excel(file_path_sabr, sheet_name="sabr_alpha")
df_sabr_rho = pd.read_excel(file_path_sabr, sheet_name="sabr_rho")
df_sabr_nu = pd.read_excel(file_path_sabr, sheet_name="sabr_nu")

def prep_sabr_params(df):
    df = df.rename(columns = {'Unnamed: 0': 'Expiry'})
    df.set_index('Expiry', inplace=True)
    df.columns.name = 'Tenors'
    return df

df_sabr_alpha = prep_sabr_params(df_sabr_alpha)
df_sabr_rho = prep_sabr_params(df_sabr_rho)
df_sabr_nu = prep_sabr_params(df_sabr_nu)

df_sabr_alpha

Tenors,1Y,2Y,3Y,5Y,10Y
Expiry,,,,,
1Y,0.139118,0.184631,0.196825,0.178150,0.172759
5Y,0.166641,0.198692,0.199945,0.188386,0.170631
10Y,0.176668,0.192693,0.196575,0.192605,0.177379


In [14]:
## interpolation of sabr params 
expiry_range = np.arange(0.25, 10.25, 0.25) # quarterly expiry range

def sabr_params_interp(df):
    interpolated_values = {}

    for tenor in df.columns:
        interp = interpolate.interp1d(
            df.index.str.replace('Y', '').astype(float), 
            df[tenor].astype(float),
            kind='linear', 
            fill_value='extrapolate'
            )
    
        interpolated_values[tenor] = interp(expiry_range)
    return pd.DataFrame(interpolated_values, index=expiry_range)


df_sabr_alpha_interp = sabr_params_interp(df_sabr_alpha)
df_sabr_rho_interp = sabr_params_interp(df_sabr_rho)
df_sabr_nu_interp = sabr_params_interp(df_sabr_nu)
df_sabr_alpha_interp.head()

,1Y,2Y,3Y,5Y,10Y
0.25,0.133957,0.181995,0.196239,0.176230,0.173158
0.50,0.135677,0.182874,0.196434,0.176870,0.173025
0.75,0.137398,0.183752,0.196630,0.177510,0.172892
1.00,0.139118,0.184631,0.196825,0.178150,0.172759
1.25,0.140838,0.185510,0.197020,0.178789,0.172626


In [15]:
## CMS rates calculation
def IRR_0(K, m, N):
    """
    Implementation of IRR(K) function 

    Args:
    K(float): strike rate
    m(float): tenor
    N(float): number of periods
    """
    value = 1/K * ( 1.0 - 1/(1 + K/m)**(N*m) )
    return value

def IRR_1(K, m, N):
    """
    Implementation of IRR'(K) function (1st derivative)
    """
    firstDerivative = -1/K*IRR_0(K, m, N) + 1/(K*m)*N*m/(1+K/m)**(N*m+1)
    return firstDerivative

def IRR_2(K, m, N):
    """ 
    Implementation of IRR''(K) function (2nd derivative)
    """
    secondDerivative = -2/K*IRR_1(K, m, N) - 1/(K*m*m)*(N*m)*(N*m+1)/(1+K/m)**(N*m+2)
    return secondDerivative


def g_0(K):
    """ 
    Implementation of g(K) function
    """
    return K

def g_1(K):
    """
    Implementation of g'(K) function (1st derivative)
    """
    return 1.0

def g_2(K):
    """
    Implementation of g''(K) function (2nd derivative)
    """
    return 0.0

def h_0(K, m, N):
    """ 
    Implementation of h(K) function
    """
    value = g_0(K) / IRR_0(K, m, N)
    return value

def h_1(K, m, N):
    """ 
    Implementation of h'(K) function (1st derivative)
    """
    firstDerivative = (IRR_0(K, m, N)*g_1(K) - g_0(K)*IRR_1(K, m, N)) / IRR_0(K, m, N)**2
    return firstDerivative

def h_2(K, m, N):
    """ 
    Implementation of h''(K) function (2nd derivative)
    """
    secondDerivative = ((IRR_0(K, m, N)*g_2(K) - IRR_2(K, m, N)*g_0(K) - 2.0*IRR_1(K, m, N)*g_1(K))/IRR_0(K, m, N)**2 
                        + 2.0*IRR_1(K, m, N)**2*g_0(K)/IRR_0(K, m, N)**3)
    return secondDerivative

In [16]:
## IRR settled option price 
def irr_settled_option_price(F, discount_factor, K, sigma, T, m, N, swap_type):
    """
    Calculate the price of an IRR-settled swaption using the Black-76 model

    Parameters:
        F (float): forward swap rate
        discount_factor (float): discount factor
        K (float): strike
        sigma (float): volatility
        T (float): time to expiry
        m (int): number of payments per year
        N (int): Tenor
        swap_type (str): Type of swap (Payer or Receiver).

    Returns:
        float: Option price.
    """
    irr_0 = IRR_0(F, m, N)
    df_numeraire = 1  # discount factor D(t,T) = 1
    black76_model = black_76_model(F, K, sigma, df_numeraire, T)
    if swap_type == 'payer':
        option_price = black76_model.blackscholes_call()
    elif swap_type == 'receiver':
        option_price = black76_model.blackscholes_put()
    else: 
        raise NameError("Invalid swap type.")
    return discount_factor * irr_0 * option_price
    

In [17]:
Expiry = [1, 5, 10]
Tenors = [1, 2, 3, 5, 10]
cms_rate = pd.DataFrame(np.zeros((len(Expiry), len(Tenors))), index=Expiry, columns=Tenors)
cms_rate


,1,2,3,5,10
1,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0


In [18]:
for expiry in Expiry:
    for tenor in Tenors:
        F = forward_swap_rates.loc[expiry, tenor]
        T = expiry
        m = 2
        N = tenor
        K = F
        alpha = df_sabr_alpha_interp.loc[T, f"{N}Y"]
        beta = 0.9
        rho = df_sabr_rho_interp.loc[T, f"{N}Y"]
        nu = df_sabr_nu_interp.loc[T, f"{N}Y"]  
        discount_factor = 1
        first_term = g_0(F)
        integrand_receive = quad(lambda k: h_2(k, 1/m, N)*irr_settled_option_price(F, 
                                                                                   discount_factor,
                                                                                    k,
                                                                                    SABR(F,k,T,alpha, beta, rho, nu),
                                                                                    T, 
                                                                                    1/m, 
                                                                                    N, 
                                                                                    'receiver'), 
                                                                                    1e-6, # lower bound close to 0
                                                                                    F, 
                                                                                    limit=100
                                                                                    )
                                                                                    
        integrand_pay = quad(lambda k: h_2(k, 1/m, N)*irr_settled_option_price(F, 
                                                                               discount_factor, 
                                                                               k, 
                                                                               SABR(F,k,T,alpha, beta, rho, nu),
                                                                                T, 
                                                                                1/m, 
                                                                                N, 
                                                                                'payer'), 
                                                                                F, 
                                                                                F*10, # upper bound instead of np.inf
                                                                                limit=100 # subintervals
                                                                                )
        
        cms_rate.loc[expiry, tenor] = first_term + (integrand_receive[0] + integrand_pay[0])

cms_rate
        
        


,1,2,3,5,10
1,0.033175,0.034975,0.035966,0.037349,0.041890
5,0.048148,0.049263,0.050178,0.049195,0.056315
10,0.058608,0.063643,0.069204,0.072152,0.100119


In [19]:
## cms pv 
